In [1]:
import os
os.environ["HF_HOME"] = "/workspace/hf"

In [2]:
!pip install -r requirements.txt
!pip install astor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 36.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

code_seach_net = load_dataset("code_search_net")

/workspace/runpod-experiments/llama2-function-name-prediction/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/runpod-experiments/llama2-function-name-prediction/venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [116]:
python_code = code_seach_net.filter(lambda example: example["language"] == "python", num_proc=8)

In [120]:
len(python_code["train"])

412178

In [121]:
len(code_seach_net["train"])

1880853

In [86]:
import ast
import astor

def remove_docstring_and_rename_function(source_code, new_name=""):
    # Parse the source code into an AST
    tree = ast.parse(source_code)

    for node in ast.walk(tree):
        # Check if the node is a FunctionDef
        if isinstance(node, ast.FunctionDef):
            # Change the function name
            node.name = new_name

            # Check for a docstring (first expression in the function body)
            if node.body and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
                # Remove the docstring
                node.body = node.body[1:]

    # Convert the AST back to source code
    return astor.to_source(tree)

In [126]:
def remove_class_name(fc_name):
    return fc_name.split(".")[-1]

def remove_underscores(fc_name):
    return fc_name.lstrip("_")

def code_transform(code, remove_encapuslation=False):
    try:
        source_code = code["func_code_string"]
        source_code = remove_docstring_and_rename_function(source_code, new_name="x")
        function_name = remove_class_name(code["func_name"])
        if remove_encapuslation:
            function_name = remove_underscores(function_name)
        return {
            "source_code": source_code,
            "function_name": function_name,
        }
    except:
        return {
            "source_code": "",
            "function_name": "",
        }



result = python_code.map(code_transform, remove_columns=list(set(python_code["train"].column_names) - {"source_code", "function_name"}), num_proc=8)

In [128]:
result = result.filter(lambda example: example["source_code"].strip() != "" and example["function_name"].strip() != "", num_proc=8)

In [131]:
# Print out the number of examples in each split
for split in result.keys():
    print(f"{split}: {len(result[split])}")

train: 408167
test: 21948
validation: 22817


In [135]:
print(result["train"][0]["source_code"])
print(result["train"][0]["function_name"])

def x(self):
    import scipy
    start = self._start_time
    seg = 1 - self.segmentation.astype(np.int8)
    self.stats['low level object voxels'] = np.sum(seg)
    self.stats['low level image voxels'] = np.prod(seg.shape)
    seg_border = scipy.ndimage.filters.laplace(seg, mode='constant')
    logger.debug('seg_border: %s', scipy.stats.describe(seg_border, axis=None))
    seg_border[seg_border != 0] = 1
    logger.debug('seg_border: %s', scipy.stats.describe(seg_border, axis=None))
    boundary_dilatation_distance = self.segparams[
        'boundary_dilatation_distance']
    seg = scipy.ndimage.morphology.binary_dilation(seg_border, np.ones([
        boundary_dilatation_distance * 2 + 1, boundary_dilatation_distance *
        2 + 1, boundary_dilatation_distance * 2 + 1]))
    if self.keep_temp_properties:
        self.temp_msgc_lowres_discontinuity = seg
    else:
        self.temp_msgc_lowres_discontinuity = None
    if self.debug_images:
        import sed3
        pd = sed3.sed3(

In [137]:
# save the dataset to disk
result.save_to_disk("python_code")

Saving the dataset (1/1 shards): 100%|██████████| 22817/22817 [00:00<00:00, 77842.62 examples/s]
